In [69]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [70]:
def sliding_window(df,window_size,first_candle): 

    df = df.sort_index(ascending=False)
    df = df.iloc[first_candle: first_candle + window_size]
    return df


In [ ]:
#Preparação dos dados:
df = pd.read_csv('dados_eurusd_1_12_2023.csv')
print(df.describe())
print(df.isnull().sum())

In [ ]:
saldo = 10000

In [ ]:
# Geração de dados
df = pd.read_csv('dados_eurusd_1_12_2023.csv')
df['action'] = np.random.choice(['buy', 'sell', 'hold'], size=len(df))
df['profit'] = np.random.uniform(-1, 1, len(df))

# Divida os dados em treinamento e teste
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)
"""
# Prepare os dados para treinamento
def prepare_data(df, window_size):
    data = []
    actions = []
    profits = []
    for i in range(len(df) - window_size):
        window = df.iloc[i:i + window_size][['open', 'high', 'close', 'low']].values
        action = df.iloc[i + window_size]['action']
        profit = df.iloc[i + window_size]['profit']
        data.append(window)
        actions.append(action)
        profits.append(profit)
    return np.array(data), np.array(actions), np.array(profits)"""

# Configurações de tamanho
window_size = 8600  # Tamanho da janela de dados
batch_size = 16  # Tamanho do lote

# Prepare os dados
X_train, y_train, profits_train = prepare_data(train_data, window_size)
X_test, y_test, profits_test = prepare_data(test_data, window_size)

# Converta rótulos para representação one-hot
"""y_train_onehot = pd.get_dummies(y_train).values
y_test_onehot = pd.get_dummies(y_test).values"""

# Define the CNN-GRU model architecture (example from a previous response)
model = tf.keras.Sequential([
    layers.Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(window_size, 1)),
    layers.MaxPooling1D(pool_size=2),
    layers.GRU(units=64, return_sequences=True),
    layers.GRU(units=32),
    layers.Dense(3, activation='softmax')  # Assuming 3 output classes
])

# Compile the model with an optimizer that handles backpropagation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Example optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (backpropagation happens within `model.fit()`)
model.fit(x_train, y_train, epochs=10, batch_size=32)

print(type(X_test))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento usando janelas deslizantes com tamanho de lote menor
for i in range(window_size, len(X_train), batch_size):
    X_window_batch = X_train[i - window_size:i]
    y_label_batch = y_train_onehot[i - window_size:i]  # Ajuste aqui
    profit_batch = profits_train[i:i + batch_size]

    # Treinamento normal
    loss, accuracy = model.train_on_batch(X_window_batch, y_label_batch)
    print(f'Batch {i//batch_size + 1}/{len(X_train)//batch_size} - Loss: {loss:.4f} - Accuracy: {accuracy:.4f}')

    # Calcule a recompensa/punição com base no lucro
    for j in range(batch_size):
        if y_train[i + j] == 'buy':
            reward = profit_batch[j]  # Lucro ao comprar
        elif y_train[i + j] == 'sell':
            reward = -profit_batch[j]  # Perda ao vender
        else:
            reward = 0.0  # Nenhuma ação, nenhuma recompensa/punição

        # Atualize os pesos usando a recompensa calculada
        loss, accuracy = model.train_on_batch(X_window_batch[j:j + 1], y_label_batch[j:j + 1], sample_weight=np.array([reward]))

    # Libere a memória após o uso
    del X_window_batch, y_label_batch, profit_batch

# Avalie o modelo
accuracy = model.evaluate(X_test, y_test_onehot)[1]
print(f'Acurácia do modelo: {accuracy}')


In [ ]:
#feito pela blackbox
import numpy as np

def calculate_profit(model, X, y, initial_capital):
    """
    Calculate the cumulative profit for each trade type.

    Args:
        model (keras.models.Model): The trained neural network model.
        X (numpy.array): The input data for the model.
        y (numpy.array): The actual output data.
        initial_capital (float): The initial capital used for the calculation.

    Returns:
        dict: A dictionary containing the cumulative profit for each trade type.
    """
    # Get the model's prediction
    prediction = model.predict(X)

    # Calculate the cumulative profit for each trade type
    profit = {'buy': 0, 'wait': 0, 'sell': 0}
    capital = initial_capital
    for i in range(len(prediction)):
        trade = np.argmax(prediction[i])
        actual_trade = np.argmax(y[i])

        # Update the profit for the actual trade type
        if trade == actual_trade:
            profit[trade] += capital
        else:
            profit[trade] -= capital

        # Update the capital for the next trade
        capital = profit[trade]

    return profit

def train_model(model, X, y, epochs, batch_size, initial_capital):
    """
    Train the model and calculate the cumulative profit for each trade type.

    Args:
        model (keras.models.Model): The trained neural network model.
        X (numpy.array): The input data for the model.
        y (numpy.array): The actual output data.
        epochs (int): The number of times the model will see the entire dataset.
        batch_size (int): The number of training examples utilized in one iteration.
        initial_capital (float): The initial capital used for the calculation.

    Returns:
        dict: A dictionary containing the cumulative profit for each trade type.
    """
    for epoch in range(epochs):
        # Train the model for one epoch
        model.fit(X, y, batch_size=batch_size, epochs=1, verbose=0)

        # Calculate the cumulative profit for each trade type
        profit = calculate_profit(model, X, y, initial_capital)

        # Print the cumulative profit for each trade type
        print(f"Epoch {epoch + 1}/{epochs}")
        for trade, value in profit.items():
            print(f" Profit for {trade}: {value}")

        # Reset the cumulative profit for each trade type
        profit = {'buy': 0, 'wait': 0, 'sell': 0}

    return profit